# Election présidentielle 2022 / 1er tour (par régions): Clustering

In [124]:
import numpy as np
import pandas as pd
import scipy.stats as sp

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import decomposition

## Données

In [2]:
data = pd.read_excel('data/resultats-par-niveau-reg-t1-france-entiere.xlsx')
print("Le jeu de données contient", data.shape[0], "lignes et", data.shape[1], "colonnes.")

Le jeu de données contient 18 lignes et 89 colonnes.


In [3]:
data.columns

Index(['Code de la région', 'Libellé de la région', 'Etat saisie', 'Inscrits',
       'Abstentions', '% Abs/Ins', 'Votants', '% Vot/Ins', 'Blancs',
       '% Blancs/Ins', '% Blancs/Vot', 'Nuls', '% Nuls/Ins', '% Nuls/Vot',
       'Exprimés', '% Exp/Ins', '% Exp/Vot', 'Sexe', 'Nom', 'Prénom', 'Voix',
       '% Voix/Ins', '% Voix/Exp', 'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25',
       'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29',
       'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45',
       'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48', 'Unnamed: 49',
       'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52', 'Unnamed: 53',
       'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56', 'Unnamed: 57',
       'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60', 'Unnamed: 61',
       'Unnamed:

### Description des données

* Les colonnes [0:4] contiennent des informations administratives sur une circonscription électorale.
* Les colonnes [4:17] contiennent des informations sur la manière dont cette circonscription électorale a voté.
* A partir de la colonne 17, chaque quatre colonnes indiquent, respectivement, le sexe, le nom, le prénom d'un candidat, et le nombre de voix, la proportion des voix sur le nombre des inscrits, la proportion des voix sur le nombre des voix exprimées. 

Nous allons, alors, modifier la dimension des données de telle sorte qu'on ait une combinaison d'une région et un candidat par ligne. 

In [4]:
n = data.shape[1]

candidats_list = list()
for i in range(17, n, 6):
    # à partir de la colonne 17, chaque 6 colonnes représentent un 
    # candidat de l'élection
    
    # pour chaque candidat, on récupère les données de la région et 
    # les votes pour le candidat dans cette région
    df_candidat = pd.concat((data.iloc[:, :17], data.iloc[:, i:i+6]), axis=1)
    
    if i > 17:
        # si  i == 17 nous sommes dans la première itération
        # sinon, nous renommons les colonnes de df_candidat 
        # de la même manière que la DataFrame du 1er candidat
        new_column_names = {
            df_candidat.columns[i]: candidats_list[0].columns[i]
            for i in range(17, len(df_candidat.columns))
        }
        
        df_candidat = df_candidat.rename(columns=new_column_names)        
    
    # nous ajoutons df_candidat à la liste des DataFrames des 
    # candidats
    candidats_list.append(df_candidat)

# nous concaténons toutes les DataFrames des candidats dans une seule
# DataFrame pour la suite de l'étude
df = pd.concat(candidats_list, axis=0, ignore_index=True)
print("Le jeu de données contient désormais", df.shape[0], "lignes et", df.shape[1], "colonnes.")

Le jeu de données contient désormais 216 lignes et 23 colonnes.


## Préparation des données

Nous commençons, tout d'abord, par la suppression les colonnes constantes (avec une seule valeur unique). 

In [5]:
df.dtypes.value_counts()

float64    10
int64       8
object      5
dtype: int64

### Variables `object`

In [6]:
df.describe(include="object").T

,count,unique,top,freq
Libellé de la région,216,18,Grand Est,12
Etat saisie,216,1,Complet,216
Sexe,216,2,M,144
Nom,216,12,ARTHAUD,18
Prénom,216,12,Nathalie,18


Nous remarquons que la variable `Etat saisie` contient une valeur unique pour toutes les lignes du jeu de données. Nous allons, alors, supprimer cette variable, puisqu'elle n'est pas informative.

In [7]:
df = df.drop("Etat saisie", axis=1)

### Variables `int64`

In [8]:
df.describe(include="int64").T

,count,mean,std,min,25%,50%,75%,max
Code de la région,216.0,3.938889e+01,3.236215e+01,1.0,6.00,30.0,75.00,94.0
Inscrits,216.0,2.602711e+06,2.066640e+06,92187.0,316023.00,2487905.0,4255350.00,7349284.0
Abstentions,216.0,6.434765e+05,4.655012e+05,55024.0,174592.00,567109.0,962499.00,1760727.0
Votants,216.0,1.959234e+06,1.607767e+06,37163.0,152403.00,1920796.0,3109141.00,5588557.0
Blancs,216.0,2.982917e+04,2.305618e+04,825.0,4001.00,30415.5,42945.00,76616.0
Nuls,216.0,1.345856e+04,9.215950e+03,553.0,3627.00,12433.0,24270.00,27967.0
Exprimés,216.0,1.915947e+06,1.576079e+06,35050.0,147372.00,1878491.5,3041926.00,5483974.0
Voix,216.0,1.596622e+05,2.763512e+05,206.0,9309.25,45273.0,130929.25,1658601.0


Les variable `Code de la région` et `Libellé de la région` contiennent la même information. Nous allons, alors, virer `Libellé de la région`, mais nous gardons un dictionnaire pour l'interprétation des résultats après la modélisation. 

In [9]:
code2libelle = {
    data["Code de la région"].iloc[i] :data["Libellé de la région"].iloc[i] 
    for i in range(data.shape[0])
}
df = df.drop("Libellé de la région", axis=1)

Les variables restantes concernent la nature et le nombre des votes pour chaque régione et chaque candidat. Nous allons nous baser su ces variables dans notre clustering. 

### Variables `float64`

In [10]:
df.describe(include="float64").T

,count,mean,std,min,25%,50%,75%,max
% Abs/Ins,216.0,33.611111,14.951546,21.21,22.9200,25.555,46.3600,63.84
% Vot/Ins,216.0,66.388889,14.951546,36.16,53.6400,74.445,77.0800,78.79
% Blancs/Ins,216.0,1.122778,0.155236,0.80,1.0100,1.100,1.2600,1.37
% Blancs/Vot,216.0,1.773889,0.434830,1.33,1.4800,1.655,1.9200,3.08
% Nuls/Ins,216.0,0.681111,0.292509,0.38,0.4800,0.555,1.0000,1.29
% Nuls/Vot,216.0,1.190556,0.823028,0.50,0.6500,0.750,1.6000,3.20
% Exp/Ins,216.0,64.585556,15.086044,34.82,51.3300,72.815,75.1700,77.03
% Exp/Vot,216.0,97.036667,1.196926,94.31,96.3000,97.670,97.8400,98.13
% Voix/Ins,216.0,5.382500,6.865727,0.19,0.7200,1.935,6.0525,25.26
% Voix/Exp,216.0,8.333519,10.939541,0.31,1.4375,2.850,12.8350,56.16


Les variables `float64` sont obtenues par des opérations sur les variables `int64`. Nous allons nous concentrer, dans notre étude, sur une modélisation avec ces variables quantitatives, sans utiliser les variables catégorielles (ou `ìnt64`).

In [11]:
code = df["Code de la région"].copy()
df = pd.concat((code, df.select_dtypes(exclude="int")), ignore_index=True, axis=1)

In [12]:
idx = dict()
for col in df.columns[df.dtypes=="object"]:
    codes, uniques = pd.factorize(df[col])
    idx[col] = uniques
    df[col] = codes

In [13]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,44,26.10,73.90,1.09,1.48,0.48,0.65,72.32,97.87,0,0,0,0.48,0.67
1,75,21.57,78.43,1.20,1.53,0.61,0.77,76.63,97.70,0,0,0,0.41,0.54
2,84,22.10,77.90,1.26,1.62,0.46,0.59,76.18,97.79,0,0,0,0.42,0.55
3,27,22.92,77.08,1.32,1.72,0.58,0.76,75.17,97.52,0,0,0,0.53,0.71
4,53,21.21,78.79,1.24,1.58,0.52,0.66,77.03,97.76,0,0,0,0.51,0.66


## Modélisation par GMM

In [129]:
def run_em(G, y, n_iter, eps=1e-7):
    n, d = y.shape
    
    mu = np.eye(G, d)
    t_y = np.eye(G, n)
    
    tau = np.ones(G) / G
    sigma = np.stack(list(np.eye(d) for _ in range(G)))
    for _ in range(n_iter):
        # E Step
        for g in range(G):
            for i in range(n):
                t_y[g,i] = tau[g] * sp.multivariate_normal.pdf(y[i], mean=mu[g], cov=sigma[g, :,: ])
            t_y[g, :] = t_y[g, :] / np.sum(t_y[g, :])
            
        # M Step
        for g in range(G):
            tau[g] = np.mean(t_y[g, :])
            
        for g in range(G):
            mu[g] = np.sum(t_y[[g],: ].T* y, axis=0) / np.sum(t_y[g, :])
        
        for g in range(G):
            sigma[g, :, :] = eps
            for i in range(n):
                y_mu_diff = (mu[g]-y[i]).reshape(-1, 1)
                sigma[g, :, :] = sigma[g, :, :] + t_y[g, i] * np.dot(y_mu_diff,  y_mu_diff.T)
            sigma[g, :, :] = sigma[g, :, :] / np.sum(t_y[g])
            

    return tau, mu, T

In [ ]:
tau, mu, T = run_em(4, np.random.uniform(size=(100, 1)), 1000)

In [ ]:
mu